In [1]:
# =====================================================================================
# ENVIRONMENT
# =====================================================================================

import snman
from snman.constants import *
import geopandas as gpd
import networkx as nx
from snman import osmnx_customized as oxc

PERIMETER = 'zrh_north-west'
# set SAVE_TO_DEBUG = True for saving the cached network into the _debug folder
# which is automatically used in the QGIS files
SAVE_TO_DEBUG = True
INTERSECTION_TOLERANCE = 10

# set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
if SAVE_TO_DEBUG:
    process_path = data_directory + 'process/' + '_debug' + '/'
else:
    process_path = data_directory + 'process/' + PERIMETER + '/'

oxc.settings.useful_tags_way = OSM_TAGS

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load perimeters')
perimeters = snman.io.load_perimeters(inputs_path + 'perimeters/perimeters.shp')

print('Get data from OSM server')
# At this step, simplification means only removing degree=2 edges
Gm = oxc.graph_from_polygon(
    # set the perimeter here
    perimeters.to_crs(4326).loc[PERIMETER]['geometry'],
    custom_filter=snman.constants.OSM_FILTER,
    simplify=True, simplify_strict=False, retain_all=True, one_edge_per_direction=False
)

print('Load manual intersections')
# polygons used to override the automatically detected intersections in some situations
given_intersections_gdf = snman.io.load_intersections(
    inputs_path + 'intersection_polygons/intersection_polygons.shp'
)

print('Load rebuilding regions')
# polygons that define which streets will be rebuilt
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    inputs_path + 'rebuilding_regions/rebuilding_regions.gpkg'
)

print('Prepare graph')
snman.osm_graph.prepare_graph(Gm)

print('Convert CRS of street graph to 2056')
snman.centerline_graph.convert_crs_of_centerline_graph(Gm, CRS)

print('Generate lanes')
# interpreting the OSM tags into a collection of lanes on each edge
snman.lane_config.generate_lanes(Gm)

print('Load sensors and assign them to edges in the raw street graph')
sensors_df = snman.io.load_sensors(inputs_path + 'sensors/sensors.csv')
snman.enrichment.match_sensors(Gm, sensors_df)

Load perimeters
Get data from OSM server
Load manual intersections
Load rebuilding regions
Prepare graph
Convert CRS of street graph to 2056
Generate lanes
Load sensors and assign them to edges in the raw street graph


In [3]:
# =====================================================================================
# ENRICH
# =====================================================================================

if 1:
    print('Identify hierarchy')
    # split the edges into hierarchy categories, such as main roads, local roads, etc.
    snman.hierarchy.add_hierarchy(Gm)

Identify hierarchy


In [4]:
# =====================================================================================
# CONSOLIDATE PARALLEL AND CONSECUTIVE EDGES
# =====================================================================================
if 1:
    print('Simplify edge geometries')
    snman.simplification.simplify_edge_geometries(Gm, 5)

for i in range(4):

    print('ITERATION', i)

    print('Detect intersections')
    intersections_gdf = snman.simplification.merge_nodes_geometric(
        Gm, INTERSECTION_TOLERANCE,
        given_intersections_gdf=given_intersections_gdf
    )

    print('Split through edges in intersections')
    snman.simplification.split_through_edges_in_intersections(Gm, intersections_gdf)

    print('Add layers to nodes')
    snman.simplification.add_layers_to_nodes(Gm)

    snman.io.export_street_graph(Gm, process_path + 'test_edges.gpkg', process_path + 'test_nodes.gpkg')

    print('Add connections between components in intersections')
    snman.simplification.connect_components_in_intersections(Gm, intersections_gdf, separate_layers=True)

    print('Consolidate intersections')
    Gm = snman.simplification.consolidate_intersections(
        Gm, intersections_gdf,
        reconnect_edges=True
    )

    print('Merge consecutive edges')
    snman.merge_edges.merge_consecutive_edges(Gm)

    print('Merge parallel edges')
    snman.merge_edges.merge_parallel_edges(Gm)

    print('Update precalculated attributes')
    snman.centerline_graph.update_precalculated_attributes(Gm)

snman.simplification.simplify_edge_geometries(Gm, 35)


Simplify edge geometries
ITERATION 0
Detect intersections
Split through edges in intersections
Add layers to nodes
Add connections between components in intersections
Consolidate intersections
Merge consecutive edges
Merge parallel edges


C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\shapely\linear.py:208: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)


Update precalculated attributes
ITERATION 1
Detect intersections
Split through edges in intersections
not len(nodes) == len(split_points) + 2 == len(edge_linestrings) + 1
[(123, <POINT (2680725.448 1251709.991)>), (1945, <POINT (2680892.615 1251706.411)>), (1946, <POINT (2680893.918 1251705.929)>), (124, <POINT (2680828.001 1251732.35)>)]
[<POINT (2680893.637 1251706.033)>]
[<LINESTRING (2680725.448 1251709.991, 2680891.686 1251706.755)>, <LINESTRING (2680893.544 1251706.068, 2680893.731 1251705.998)>, <LINESTRING (2680891.686 1251706.755, 2680890.948 1251715.429, 2680872.89 12...>]
not len(nodes) == len(split_points) + 2 == len(edge_linestrings) + 1
[(157, <POINT (2682509.853 1251794.461)>), (1971, <POINT (2682467.329 1251835.764)>), (1972, <POINT (2682464.925 1251835.235)>), (615, <POINT (2682473.897 1251907.568)>)]
[<POINT (2682465.218 1251835.299)>]
[<LINESTRING (2682509.853 1251794.461, 2682481.842 1251808.026, 2682469.343 1...>, <LINESTRING (2682465.315 1251835.32, 2682465.12 125

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\shapely\linear.py:208: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)


Update precalculated attributes
ITERATION 2
Detect intersections
Split through edges in intersections
not len(nodes) == len(split_points) + 2 == len(edge_linestrings) + 1
[(9, <POINT (2680703.601 1251756.205)>), (2404, <POINT (2680674.98 1251756.532)>), (2405, <POINT (2680675.301 1251754.757)>), (1844, <POINT (2680646.16 1251750.173)>)]
[<POINT (2680675.248 1251755.052)>]
[<LINESTRING (2680703.601 1251756.205, 2680674.729 1251757.913)>, <LINESTRING (2680675.23 1251755.15, 2680675.266 1251754.954)>, <LINESTRING (2680674.729 1251757.913, 2680646.391 1251749.885, 2680646.295 1...>]
not len(nodes) == len(split_points) + 2 == len(edge_linestrings) + 1
[(123, <POINT (2680725.448 1251709.991)>), (2473, <POINT (2680892.615 1251706.411)>), (2474, <POINT (2680893.918 1251705.929)>), (2475, <POINT (2680828.09 1251732.624)>), (2476, <POINT (2680831.676 1251743.632)>), (807, <POINT (2680833.817 1251755.246)>)]
[<POINT (2680893.637 1251706.033)> <POINT (2680828.09 1251732.624)>
 <POINT (2680831.678 

In [5]:
print('Add lane stats to edges')
# how many lanes, how wide, etc.
snman.lane_config.generate_lane_stats(Gm)

Add lane stats to edges


In [6]:
# =====================================================================================
# ENRICH
# =====================================================================================

if 0:
    #TODO: use mapmatching for better performance and accuracy
    #TODO: add route direction for one-way sections
    print('Add public transport')
    pt_network = snman.io.import_geofile_to_gdf(inputs_path + "public_transit/ZVV_LINIEN_GEN_L.shp")
    snman.enrichment.match_pt(Gm, pt_network)

if 1:
    print('Update OSM tags')
    # to match the simplified and merged edges
    snman.lane_config.update_osm_tags(Gm)

if 1:
    print('Add elevation')
    spn = oxc.stats.count_streets_per_node(Gm, nodes=Gm.nodes)
    nx.set_node_attributes(Gm, values=spn, name="street_count")
    Gm = oxc.elevation.add_node_elevations_raster(Gm, inputs_path + 'ch_dhm_25/2056/ch_dhm_2056.tif', cpus=1)
    Gm = oxc.elevation.add_edge_grades(Gm, add_absolute=False)

if 0:
    print('Add traffic counts')
    source = gpd.read_file(inputs_path + 'traffic_volumes/npvm_2017_filtered.gpkg').to_crs(2056)
    source['fid'] = source.index
    # Remove links with zero traffic (otherwise they will distort the averages on the matched links)
    source = source[source['DTV_ALLE'] > 0]
    snman.enrichment.match_linestrings(Gm, source, [
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_avg',         'agg': 'avg' },
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_max',         'agg': 'max' },
        {'source_column': 'FROMNODENO', 'target_column': 'npvm_fromnodeno', 'agg': 'list'},
        {'source_column': 'TONODENO',   'target_column': 'npvm_tonodeno',   'agg': 'list'}
    ])


Update OSM tags
Add elevation


In [7]:
# =====================================================================================
# EXPORT
# =====================================================================================

if 1:
    print('Export street graph graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.io.export_street_graph(Gm, process_path + 'street_graph_edges.gpkg', process_path + 'street_graph_nodes.gpkg')

if 1:
    print('Export lane geometries')
    # each lane has an own geometry and with as an attribute, for visualization purposes
    snman.io.export_street_graph_with_lanes(Gm, KEY_LANES_DESCRIPTION, process_path + 'lane_geometries.shp', scaling=2)

if 1:
    print('Save intersection geometries into a file')
    snman.io.export_gdf(intersections_gdf, process_path + 'intersections_polygons.gpkg', columns=['geometry'])

if 0:
    print('Save raw OSM graph')
    snman.io.export_street_graph(Gm_raw, process_path + 'raw_street_graph_edges.gpkg', process_path + 'raw_street_graph_nodes.gpkg')

if 0:
    print('Save raw OSM graph with connectors')
    snman.io.export_street_graph(Gm_unsimplified,
        process_path + 'raw_osm_graph_with_connectors_edges.gpkg',
        process_path + 'raw_osm_graph_with_connectors_nodes.gpkg'
    )

Export street graph graph
Export lane geometries
Save intersection geometries into a file
